<a href="https://colab.research.google.com/github/DiegoVena/MBA_aula_de_spark/blob/main/Configura%C3%A7%C3%A3o_spark_atividade_03_MBA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#configurando o ambiente 
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark


In [2]:
#criando as pastas do Java e Spark
import os
os.environ['JAVA_HOME']="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['SPARK_HOME']='/content/spark-3.1.2-bin-hadoop2.7'

In [3]:
!pip install -q findspark

In [4]:
#Esse pacote referência o spark
import findspark

In [5]:
findspark.init()

In [6]:
#importa o spark sessão
from pyspark.sql import SparkSession

In [7]:
#Configuração da sessão
spark= SparkSession.builder \
    .master('local[*]') \
    .appName('Aula_spark') \
    .config('spark.iu.port','4050') \
    .getOrCreate()

In [8]:
#!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip

In [9]:

#get_ipython().system_raw('ngrok config add-authtoken 2PieZgGtQEXUx4JnerL86qjMTaI_23TviCWZVtdBFfJ4NGGj')
#get_ipython().system_raw('./ngrok http 4050 &')*/

In [10]:
#!curl -s http://localhost:4050/api/tunnels

In [11]:
#criando uma lista de tupla
data=[('Diego',26),('Josi',23)]

In [12]:
#criando uma lista de colunas
Col=['Nome','Idade']

In [13]:
#Criando um data frame no spark
df=spark.createDataFrame(data,Col)

In [14]:
#Mostrando o data frame 
df.show()

+-----+-----+
| Nome|Idade|
+-----+-----+
|Diego|   26|
| Josi|   23|
+-----+-----+



In [15]:
#Monstrando o data frame e modelo de Data frame pandas(py)
df.toPandas()

,Nome,Idade
0,Diego,26
1,Josi,23


In [16]:
#criando uma conexão com o Drive caso queira se não desconsidere esse passo 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
#biblioteca de descompactação
import zipfile

In [18]:
#extraindo o arquivo do zip para a mesma pasta 
zipfile.ZipFile('/content/drive/MyDrive/MBA_03/empresas.zip','r').extractall('/content/drive/MyDrive/MBA_03')

In [19]:
#criando um caminho
path='/content/drive/MyDrive/MBA_03/empresas'

In [20]:
#lendo csv 
empresas=spark.read.csv('/content/drive/MyDrive/MBA_03/empresas/part-00000-58983ad4-8444-4405-aec6-9cd3e5413d1b-c000.csv',sep=';',inferSchema=True)

In [21]:
#mostrar as 20 primeiras linhas, dentro do parentes de show podemos escolher quantas linhas vamos
#mostrar igual a função head do python
empresas.show()

+-----+--------------------+----+---+---------+---+----+
|  _c0|                 _c1| _c2|_c3|      _c4|_c5| _c6|
+-----+--------------------+----+---+---------+---+----+
| 4519|DANIELA DA SILVA ...|2135| 50|     0,00|  5|null|
| 8638|JOAO DOS SANTOS F...|2135| 50|     0,00|  5|null|
|11748|PANIFICADORA E CO...|2062| 49|     0,00|  1|null|
|12027| L G SORVETERIA LTDA|2062| 49|     0,00|  5|null|
|13289|ANDREIA CRISTINA ...|2305| 65|100000,00|  1|null|
|13623|MARISTELA INDUSTR...|2062| 49|     0,00|  5|null|
|17389|DAICICOM MARKETIN...|2240| 49|     0,00|  1|null|
|18944|SAO GOTARDO-DISTR...|2062| 49|     0,00|  5|null|
|19204|TORTARIA CAMPINAS...|2062| 49|     0,00|  1|null|
|22223|S R FARIAS DA SIL...|2062| 49|     0,00|  5|null|
|23015|EVANGELINA P DE J...|2135| 50|     0,00|  1|null|
|24354|JUCELIA PEREIRA D...|2135| 50|     0,00|  1|null|
|26708|DONEIR RODRIGUES ...|2135| 50|     0,00|  1|null|
|28664|M ROCHA COML IMPO...|2062| 49|100000,00|  5|null|
|28759|LULETEF CONFECCAO...|206

In [22]:
#mostrando a quantidade de linhas no Data frame
empresas.count()

458549

In [23]:
#criando uma lista com nome da colunas
Names=['cnpj_basico','razao_social','Natureza_juridica','qualificacao_responsavel','capital_social','porte_empresa','ente_federativo_responsável']

In [24]:
#renomeando as colunas
for index, ColNames in enumerate(Names):
  empresas=empresas.withColumnRenamed(f"_c{index}",ColNames)


In [25]:
#Colunas após a transformação
empresas.columns

['cnpj_basico',
 'razao_social',
 'Natureza_juridica',
 'qualificacao_responsavel',
 'capital_social',
 'porte_empresa',
 'ente_federativo_responsável']

In [26]:
#Vendo o tipo de dados dentro do data frame
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- Natureza_juridica: integer (nullable = true)
 |-- qualificacao_responsavel: integer (nullable = true)
 |-- capital_social: string (nullable = true)
 |-- porte_empresa: integer (nullable = true)
 |-- ente_federativo_responsável: string (nullable = true)



In [27]:
#importando double,string e Datatype
from pyspark.sql.types import DoubleType, StringType,DateType

In [28]:
# importando funções matemáticas
from pyspark.sql import functions as f

In [29]:
#Mudando ',' para '.'
empresas=empresas.withColumn('capital_social',f.regexp_replace('capital_social',',','.'))

In [30]:
empresas.show( )



+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|cnpj_basico|        razao_social|Natureza_juridica|qualificacao_responsavel|capital_social|porte_empresa|ente_federativo_responsável|
+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|       4519|DANIELA DA SILVA ...|             2135|                      50|          0.00|            5|                       null|
|       8638|JOAO DOS SANTOS F...|             2135|                      50|          0.00|            5|                       null|
|      11748|PANIFICADORA E CO...|             2062|                      49|          0.00|            1|                       null|
|      12027| L G SORVETERIA LTDA|             2062|                      49|          0.00|            5|                       null|
|      13289|ANDREIA CRISTINA ...|             2305|   

In [31]:
#Transformando string para Double
empresas=empresas.withColumn('capital_social',empresas['capital_social'].cast(DoubleType()))

In [32]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- Natureza_juridica: integer (nullable = true)
 |-- qualificacao_responsavel: integer (nullable = true)
 |-- capital_social: double (nullable = true)
 |-- porte_empresa: integer (nullable = true)
 |-- ente_federativo_responsável: string (nullable = true)



In [33]:
empresas \
  .select('*') \
  .show(5,False)

+-----------+---------------------------------------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|cnpj_basico|razao_social                                       |Natureza_juridica|qualificacao_responsavel|capital_social|porte_empresa|ente_federativo_responsável|
+-----------+---------------------------------------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|4519       |DANIELA DA SILVA CRUZ                              |2135             |50                      |0.0           |5            |null                       |
|8638       |JOAO DOS SANTOS FAGUNDES                           |2135             |50                      |0.0           |5            |null                       |
|11748      |PANIFICADORA E CONFEITARIA CONFIANCA RIO PRETO LTDA|2062             |49                      |0.0           |1            |null                       |
|120

In [34]:
#usando filtro where

empresas \
  .where('capital_social>20 and capital_social<100') \
  .count()

839

In [35]:
# Usando filtro 'filter'
empresas \
  .filter('capital_social>20 and capital_social<100') \
  .show()

+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|cnpj_basico|        razao_social|Natureza_juridica|qualificacao_responsavel|capital_social|porte_empresa|ente_federativo_responsável|
+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|   14715041|LIDIANE MARIA DO ...|             2135|                      50|          50.0|            1|                       null|
|   17839609|RENATO PEREIRA DA...|             2135|                      50|          32.0|            1|                       null|
|   18953382|ANA BEATRIZ MACHA...|             2135|                      50|          40.0|            1|                       null|
|   20601885|CRISTIANO AKIHITO...|             2135|                      50|          50.0|            1|                       null|
|   23023326|DIEGO DA SILVA SE...|             2135|   

In [36]:
#usando comando SQL no spark
empresas \
  .select('razao_social','porte_empresa') \
  .where('capital_social>20 and capital_social<100') \
  .show(5)

+--------------------+-------------+
|        razao_social|porte_empresa|
+--------------------+-------------+
|LIDIANE MARIA DO ...|            1|
|RENATO PEREIRA DA...|            1|
|ANA BEATRIZ MACHA...|            1|
|CRISTIANO AKIHITO...|            1|
|DIEGO DA SILVA SE...|            1|
+--------------------+-------------+
only showing top 5 rows



In [37]:
#orderBy
empresas \
  .select('razao_social','porte_empresa') \
  .where('capital_social>20 and capital_social<100') \
  .orderBy('razao_social',ascending =True) \
  .show(5,True)


+--------------------+-------------+
|        razao_social|porte_empresa|
+--------------------+-------------+
|ADAO MARTINS VIEI...|            1|
|ADEMAR ALVES VIEI...|            1|
|ADEMIR AGUIAR PER...|            1|
|ADEMISIA BRITO MA...|            1|
|ADENILCE RIBEIRO ...|            1|
+--------------------+-------------+
only showing top 5 rows



In [38]:
#sumarizando os dados com a função agg
# Qual a média de capital social//porte? 
empresas \
    .select('*')\
    .groupBy('porte_empresa')\
    .agg(
        f.avg('capital_social')
    ) \
    .orderBy('porte_empresa', ascending=True)\
    .show()

+-------------+-------------------+
|porte_empresa|avg(capital_social)|
+-------------+-------------------+
|         null|                0.0|
|            1| 207941.52301797093|
|            3|  154639.3795564377|
|            5|  606829.3027531682|
+-------------+-------------------+



In [39]:
#quantas empresas não tem capital social//porte?
empresas \
    .select('*')\
    .filter('capital_social==0') \
    .groupBy('porte_empresa')\
    .agg(
        f.count('capital_social')
    ) \
    .orderBy('porte_empresa', ascending=True)\
    .show()

+-------------+---------------------+
|porte_empresa|count(capital_social)|
+-------------+---------------------+
|         null|                  576|
|            1|                45696|
|            3|                 1166|
|            5|               121688|
+-------------+---------------------+



In [50]:
# quantidade de natureza juridica distinta??
empresas \
   .select('Natureza_juridica') \
   .agg(
      f.countDistinct('Natureza_juridica') 
   ) \
   .show()

+------------------------+
|count(Natureza_juridica)|
+------------------------+
|                      65|
+------------------------+



In [40]:
#salvando em formato Parquet 
empresas.write.parquet(
    path='/content/drive/MyDrive/curso-spark/empresas/parquet',
    mode='overwrite'
)

In [41]:
#testando e subindo o arquivo parquet

empresas_2=spark.read.parquet('/content/drive/MyDrive/MBA_03/empresas/parquet/part-00000-d75cf41c-ca89-4849-946a-bcf8eee2a77b-c000.snappy.parquet')